In [1]:
from plotly import express as px
import pandas as pd
df = pd.read_parquet("../../data/processed/pyarrow/UfR_text.parquet")
df_unhydrated = pd.read_pickle("../../data/unhydrated_ufr.pkl")
df_unhydrated = df_unhydrated.loc[df_unhydrated["document_category"]!="Artikler"]


In [2]:
df = pd.merge(df.loc[:,["id_verdict","year"]], df_unhydrated.loc[:,["id","document_category"]],left_on="id_verdict",right_on="id")
df.rename(columns={"document_category":"Court"},inplace=True)
df.drop(columns="id",inplace=True)
df.drop_duplicates("id_verdict",inplace=True)

In [3]:
df.loc[df["Court"] == "", "Court"] = "Ingen kategori"
df["year"] = df["year"].astype(int)
court_types = ['Andre / ophørte domstole','Andre myndigheder og styrelser','Den Særlige Klageret', 'Grosserersocietetet', "Højesteret","Vestre Landsret", "Østre Landsret",'Sø- og Handelsretten', "Byretterne",'Ingen kategori', "Rigsretten"]
court_types_rename = ["Other", "Other", "Other", "Other","Supreme Court", "Western High Court", "Eastern High Court", "Maritime and Commercial High Court", "District Courts", "Other", "Other"]
rename_dict = dict(zip(court_types,court_types_rename))

df["Court"] = df["Court"].apply(lambda x: rename_dict[x] if x in rename_dict.keys() else x)
df_gb= df.groupby(['year','Court'])['id_verdict'].count()
df_gb= df_gb.reset_index(level=1)
df_gb = df.groupby(["year","Court"]).count()
df_summary = pd.pivot_table(df_gb,index="year",columns="Court",fill_value=0)["id_verdict"]

df_summary = df_summary.loc[:,["Supreme Court", "Western High Court", "Eastern High Court", "District Courts", "Maritime and Commercial High Court", "Other"]]
fig = px.bar(df_summary,template="ggplot2",color_discrete_sequence=px.colors.qualitative.Pastel)
fig = fig.update_yaxes(title="Count of documents")
fig = fig.update_xaxes(title="Year", showgrid=False)
fig = fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    bgcolor='rgba(0,0,0,0.1)'
))
fig = fig.update_layout(margin=dict(
                        l=0,
                        r=0,
                        b=0,
                        t=0
                        ), font_family="sarif"
                        )
fig.write_html("../../data/plots/judgement_by_court.html")
fig.write_image("../../data/plots/judgement_by_court.pdf")
fig

#### Division in criminal cases

In [8]:
import re
def straffesag(s):
    if re.search(("rigsadvokat|anklagemyndighed"),s,re.IGNORECASE):
        return True
    else:
        return False

df_label = pd.read_parquet("../../data/processed/pyarrow/code_sagsomkostninger.parquet")
df_label["straffesag"] = df_label["prosecutor"].apply(straffesag)
df_m = pd.merge(df_label.loc[:,["id_verdict","LABEL_party_who_pays","omkostninger","straffesag"]],df)


In [20]:
df_m.pivot_table(index=["Court","LABEL_party_who_pays"],columns="straffesag",values="id_verdict", aggfunc='count').to_excel("../../data/tables/label_distriubtion_by_court.xlsx")

In [19]:
df_m.loc[df_m["straffesag"] & (df_m["LABEL_party_who_pays"]=="NO_COST")]

,id_verdict,LABEL_party_who_pays,omkostninger,straffesag,year,Court
8516,U.1988.950,NO_COST,[I forholdet mellem tiltalte og M betaler inge...,True,1988,Western High Court
10986,U.2003.2643,NO_COST,[Rødovre Rets dom af 7. februar 2003 - - - er ...,True,2003,Eastern High Court
12969,U.2010.79,NO_COST,[Landsretten ophæver herefter byrettens dom fo...,True,2010,Western High Court
13679,U.2013.3192,NO_COST,[Thi kendes for ret: Anklagemyndigheden frifin...,True,2013,Eastern High Court
13750,U.2013.986,NO_COST,"[Højesteret finder, at der foreligger sådanne ...",True,2013,Supreme Court
